In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-dataset/tu_tr.txt
/kaggle/input/train-dataset/kn_tr.txt
/kaggle/input/test-dataset/tu_dev.txt
/kaggle/input/test-dataset/kn_dev.txt


In [2]:
!pip install torch torchvision torchaudio

In [3]:
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
#!git checkout v0.10.2
##!python -m pip install --user ./
!pip install --editable .
%cd ..

Cloning into 'fairseq'...
remote: Enumerating objects: 35203, done.
remote: Total 35203 (delta 0), reused 0 (delta 0), pack-reused 35203
Receiving objects: 100% (35203/35203), 25.35 MiB | 32.05 MiB/s, done.
Resolving deltas: 100% (25494/25494), done.
/kaggle/working/fairseq
Obtaining file:///kaggle/working/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 15.4 MB/s eta 0:00:00
  Building editable for fairseq (

In [4]:
!nvidia-smi

Mon Jul  1 16:15:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              26W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!pip install git+https://github.com/vmujadia/tokenizer.git --upgrade

  Cloning https://github.com/vmujadia/tokenizer.git to /tmp/pip-req-build-yrdc4a9z
  Running command git clone --filter=blob:none --quiet https://github.com/vmujadia/tokenizer.git /tmp/pip-req-build-yrdc4a9z
  Resolved https://github.com/vmujadia/tokenizer.git to commit 93cd09b81702108a51c08c9796fd1cc941a1b98b
  Preparing metadata (setup.py) ... done
  Created wheel for IL-Tokenizer: filename=IL_Tokenizer-0.0.2-py3-none-any.whl size=7225 sha256=f8690efa3e944c46fa1a90c7846a7b5e8315f34a2fc27f0b29092c60a46fc4fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-gb2awdxf/wheels/9a/fb/5b/3d75bfde8561726121c09f0f0a83389c05312df8a513808c41
Successfully built IL-Tokenizer
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at h

In [6]:
!git clone https://github.com/moses-smt/mosesdecoder.git

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148459, done.
remote: Counting objects: 100% (887/887), done.
remote: Compressing objects: 100% (366/366), done.
remote: Total 148459 (delta 565), reused 797 (delta 517), pack-reused 147572
Receiving objects: 100% (148459/148459), 129.99 MiB | 24.83 MiB/s, done.
Resolving deltas: 100% (114591/114591), done.


In [7]:
!git clone https://github.com/rsennrich/subword-nmt.git

Cloning into 'subword-nmt'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 600 (delta 10), reused 12 (delta 4), pack-reused 576
Receiving objects: 100% (600/600), 253.58 KiB | 7.46 MiB/s, done.
Resolving deltas: 100% (359/359), done.


In [8]:
!ls mosesdecoder/scripts/training/clean-corpus-n.perl

mosesdecoder/scripts/training/clean-corpus-n.perl


In [9]:
# Use the datasets already downloaded in the Kaggle notebook
!cp /kaggle/input/train-dataset/kn_tr.txt train.tgt
!cp /kaggle/input/train-dataset/tu_tr.txt train.src
!cp /kaggle/input/test-dataset/kn_dev.txt valid.tgt
!cp /kaggle/input/test-dataset/tu_dev.txt valid.src


In [10]:
print ('Data Stats')
! wc -l train.*
! wc -l valid.*

Data Stats
   8300 train.src
   8300 train.tgt
  16600 total
  1000 valid.src
  1000 valid.tgt
  2000 total


In [11]:
from ilstokenizer import tokenizer
import codecs

def to_tokenize_and_lower(input_path, output_path):
  outfile = open(output_path, 'w')
  for line in codecs.open(input_path):
    line = line.strip()
    line = tokenizer.tokenize(line).lower()
    #print (line)
    outfile.write(line+'\n')
  outfile.close()

In [12]:
to_tokenize_and_lower('train.src','train.src.tkn')
to_tokenize_and_lower('train.tgt','train.tgt.tkn')

to_tokenize_and_lower('valid.src','valid.src.tkn')
to_tokenize_and_lower('valid.tgt','valid.tgt.tkn')

In [13]:
! cat train.src.tkn > train.all.tkn
! cat train.tgt.tkn >> train.all.tkn

In [14]:
! perl mosesdecoder/scripts/training/clean-corpus-n.perl -ratio 1.5 train src.tkn tgt.tkn train_filtered 1 250

clean-corpus.perl: processing train.src.tkn & .tgt.tkn to train_filtered, cutoff 1-250, ratio 1.5

Input sentences: 8300  Output sentences:  7945


In [16]:
print ('Data Stats')
! wc -l train*
! wc -l valid*

Data Stats
  16600 train.all.tkn
   8300 train.src
   8300 train.src.tkn
   8300 train.tgt
   8300 train.tgt.tkn
   7945 train_filtered.src.tkn
   7945 train_filtered.tgt.tkn
  65690 total
  1000 valid.src
  1000 valid.src.tkn
  1000 valid.tgt
  1000 valid.tgt.tkn
  4000 total


In [17]:
!python subword-nmt/subword_nmt/learn_bpe.py -s 3500 < train.all.tkn > train.codes

100%|######################################| 3500/3500 [00:07<00:00, 455.08it/s]


In [18]:
! head -n 10 train.codes

#version: 0.2
ತ ್
ಲ ್
ತ್ ತ
ಂ ದ
ನ ು</w>
ರ ್
ಲ್ ಲ
ನ ್
ಕ ್


In [19]:
!python subword-nmt/subword_nmt/apply_bpe.py -c train.codes < train.src.tkn > train.tu
!python subword-nmt/subword_nmt/apply_bpe.py -c train.codes < train.tgt.tkn > train.kn

!python subword-nmt/subword_nmt/apply_bpe.py -c train.codes < valid.src.tkn > valid.tu
!python subword-nmt/subword_nmt/apply_bpe.py -c train.codes < valid.tgt.tkn > valid.kn

In [20]:
! head -n 10 train.tu

ಬದಲಾವಣೆ ಬರೆಯ@@ ಿನ@@ ಾರ್ ಚ@@ ಂದು ಪನ್ಪ@@ ಿನ@@ ಾರ್
ದೇವೆರ್ ಕೊರ್@@ ನಗ ಮಾತ@@ ಲ ಕೊರ್ಪೆರ್
ಅಂಚನೆ ಮನ@@ ಸು@@ ಗು ಆಯಿನ ಬೇನೆ ಆ ಬೇನೆ ಮಾ@@ ಜು@@ ಜಿ
ವಿಭವೆ ನಡತೊಂದುಲ್ಲೆ
ನಮಕ್ ಸೊ@@ ಲ@@ ಬೊ@@ ಗು ಅರ್ಥ ಆವ@@ ಂದಿನ ಒಂಜಿ ವಿ@@ ಜ್ಞಾನ ಇ@@ ತ್ತ್ಂಡ್
ಈ ಪ್ರಕರ@@ ಣದ ಪತ್ತ@@ ೆಡ್ ದ.@@ ಕ ಜಿಲ್ಲಾ ಪೊಲೀಸ್ ಅಧೀ@@ ಕ್ಷ@@ ಕೆರ@@ ಾಯಿನ ಶ್ರೀ ಋ@@ ಷ@@ ಿ@@ ಕೇ@@ ಶ್ ಸೋ@@ ನಾ@@ ವಣೆ ಐ@@ .@@ ಪಿ@@ .@@ ಎಸ್ ಮೆರ್ನ ನಿರ್ದೇಶ@@ ನದ ಪ್ರಕಾರ ದ.@@ ಕ ಜಿಲ್ಲಾ ಹೆಚ್ಚ@@ ುವ@@ ರಿ ಪೊಲೀಸ್ ಅಧೀ@@ ಕ್ಷ@@ ಕೆರ@@ ಾಯಿನ ಶ್ರೀ ವಿ@@ .@@ ಬಿ ಭಾ@@ ಸ್@@ ಕರ್ ಮೆರ್ನ ಮಾರ್ಗ@@ ದರ್ಶ@@ ನ@@ ಡ್ ಬಂಟ್ವಾಳ ಉಪ@@ ವಿಭ@@ ಾಗ@@ ದ ಪೊಲೀಸ್ ಉಪಾ@@ ಧೀ@@ ಕ್ಷ@@ ಕೆರ@@ ಾಯಿನ ಶ್ರೀ ವೆ@@ ಲೈ@@ ಂಟ@@ ೈ@@ ನ್ ಡಿಸ@@ ೋ@@ ಜ@@ ಾ ಮೆರ್ನ ಮುತಾಲಿ@@ ಕೆ@@ ಡ್ ಬಂಟ್ವಾಳ ನಗರ ಠಾಣ@@ ಾ ಪೊಲೀಸ್ ನಿರೀಕ್ಷ@@ ಕೆರ@@ ಾಯಿನ ಚೆ@@ ಲ@@ ುವ@@ ರಾಜ@@ ು ಬಿ , ಬಂಟ್ವಾಳ ನಗರ ಠಾಣ@@ ಾ ಪೊಲೀಸ್ ಉಪ@@ ನಿರೀಕ್ಷ@@ ಕೆರ್ ಬೊಕ್ಕ ಸಿಬ್ಬಂದಿ@@ ಲು ಬಂಟ್ವಾಳ ಗ್ರಾ@@ ಮಾ@@ ಂತರ ಪೊಲೀಸ್ ಠಾಣ@@ ಾ ಉಪ@@ ನಿರೀಕ್ಷ@@ ಕೆರ್ ಅಂಚನೇ ಸಿಬ್ಬ@@ ಂದ@@ ಿನ@@ ಕುಲು ಬಂಟ್ವಾಳ ನಗರ ಪೊಲೀಸ್ ಠಾಣ@@ ಾ ಅಪ@@ ರಾ@@ ಧ ವಿಭ@@ ಾಗ@@ ದ ಪೊಲೀಸ್ ಉಪ@@ ನಿರೀಕ್ಷ@@ ಕೆರ@@ ಾಯಿನ ,
ತ@@ ತ್ತಿ ಕಾಲ@@ ೊಗು ಸೋ@@ ಲು
ಅ@@ ಸ್@@ ಸಾ@@ ಂ ದ ದುಂಬುದ ಮುಖ್ಯಮಂ@@ ತ್ರಿ ಸ್

In [21]:
! head -n 10 train.kn

ಬದಲಾವಣೆ ಬರೆ@@ ದವರು ಚ@@ ಂದು ಎನ್ನ@@ ುವವರು
ದೇವರು ಕೊಡ@@ ುವಾಗ ಎಲ್ಲ@@ ವ@@ ನ್ನೂ ಕೊಡುತ್ತಾರೆ
ಹಾಗೆಯೇ ಮನಸ್@@ ಸಿಗೆ ಆದ ನೋವು ಆ ನೋವು ಮಾ@@ ಸ@@ ುವುದಿಲ್ಲ
ವಿಭವನು ನಡೆಯುತ್ತಿದ್ದಾನೆ
ನಮಗೆ ಸು@@ ಲಭ@@ ವಾಗಿ ಅರ್ಥ ಆಗ@@ ದ ಒಂದು ವಿ@@ ಜ್ಞಾನ ಇತ್ತು
ಈ ಪ್ರಕರ@@ ಣದ ಪತ್ತ@@ ೆಯಲ್ಲಿ ದ.@@ ಕ . ಜಿಲ್ಲಾ ಪೆçಲೀಸ್ ಅಧೀ@@ ಕ್ಷ@@ ಕ@@ ರಾದ ಶ್ರೀ ಋ@@ ಷ@@ ಿ@@ ಕೇ@@ ಶ್ ಸೋ@@ ನಾ@@ ವಣೆ ಐ@@ .@@ ಪಿ@@ .@@ ಎಸ್ ಇವರ ನಿರ್ದೇಶ@@ ನದ ಪ್ರಕಾರ ದ.@@ ಕ@@ .@@ ಜಿಲ್ಲಾ ಹೆಚ್ಚ@@ ುವ@@ ರಿ ಪೆçಲೀಸ್ ಅಧೀ@@ ಕ್ಷ@@ ಕ@@ ರಾದ ಶ್ರೀ ವಿ@@ .@@ ಬಿ . ಭಾ@@ ಸ್@@ ಕರ್@@ ‍@@ ರ@@ ವರ ಮಾರ್ಗ@@ ದರ್ಶ@@ ನದಲ್ಲಿ ಬಂಟ್ವಾಳ ಉಪ@@ ವಿಭ@@ ಾಗ@@ ದ ಪೆçಲೀಸ್ ಉಪ@@ ಧೀ@@ ಕ್ಷ@@ ಕ@@ ರಾದ ಶ್ರೀ ವೆ@@ ಲೈ@@ ಂಟ@@ ೆನ್ ಡಿಸ@@ ೋ@@ ಜ@@ ರ@@ ವರ ಮು@@ ಂದಾ@@ ಳ@@ ತ್ವ@@ ದಲ್ಲಿ ಬಂಟ್ವಾಳ ನಗರ ಠಾಣ@@ ಾ ಪೆçಲೀಸ್ ನಿರೀಕ್ಷ@@ ಕ@@ ರಾದ ಚೆ@@ ಲ@@ ುವ@@ ರಾಜ@@ ು ಬಿ . ಬಂಟ್ವಾಳ ನಗರ ಠಾಣ@@ ಾ ಪೆçಲೀಸ್ ಉಪ@@ ನಿರೀಕ್ಷ@@ ಕರು ಮತ್ತು ಸಿಬ್ಬಂದಿ@@ ಗಳು ಬಂಟ್ವಾಳ ಗ್ರಾ@@ ಮಾ@@ ಂತರ ಪೆçಲೀಸ್ ಠಾಣ@@ ಾ ಉಪ@@ ನಿರೀಕ್ಷ@@ ಕರು ಹಾಗೂ ಸಿಬ್ಬಂದಿ@@ ಗಳು ಬಂಟ್ವಾಳ ನಗರ ಪೆçಲೀಸ್ ಠಾಣ@@ ಾ ಅಪ@@ ರಾ@@ ಧ ವಿಭ@@ ಾಗ@@ ದ ಪೆçಲೀಸ್ ಉಪ@@ ನಿರೀಕ್ಷ@@ ರಾದ
ತಪ್ಪ@@ ಿದ ಕಾಲ@@ ಕ್ಕೆ ಸೋ@@ ಲು
ಅ@@ ಸ್@@ ಸಾಂ@@ ನ ಮು@@ ಂದಿನ ಮುಖ್ಯಮಂ@@ ತ್ರಿ

In [22]:
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

/kaggle/lib/kagglegym:/kaggle/lib:/content/fairseq/


In [23]:
! python fairseq/fairseq_cli/preprocess.py \
    --joined-dictionary \
    --source-lang tu --target-lang kn \
    --trainpref train --validpref valid --testpref valid \
    --destdir data-bin/trial --thresholdtgt 0 --thresholdsrc 0 \
    --workers 20

2024-07-01 16:19:17.882343: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 16:19:17.882461: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 16:19:18.012243: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/kaggle/working/fairseq/fairseq/tasks/multires_hubert_pretraining.py:154: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  dictionaries = [ (Dictionary.load(f"{label_dir}/dict.{label}.txt") if label is not "" else None ) for label in self.cfg.labels]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.f

In [24]:
ls data-bin/trial

dict.kn.txt        test.tu-kn.kn.idx   train.tu-kn.kn.idx  valid.tu-kn.kn.idx
dict.tu.txt        test.tu-kn.tu.bin   train.tu-kn.tu.bin  valid.tu-kn.tu.bin
preprocess.log     test.tu-kn.tu.idx   train.tu-kn.tu.idx  valid.tu-kn.tu.idx
test.tu-kn.kn.bin  train.tu-kn.kn.bin  valid.tu-kn.kn.bin


In [25]:
! python fairseq/fairseq_cli/train.py --fp16 \
    data-bin/trial \
    --source-lang tu --target-lang kn \
    --arch transformer_iwslt_de_en --share-all-embeddings \
    --encoder-embed-dim	128 --encoder-ffn-embed-dim	128 \
    --encoder-layers	2 --encoder-attention-heads	2 \
    --decoder-embed-dim	128 --decoder-ffn-embed-dim	128 \
    --decoder-layers	2 --decoder-attention-heads	2 \
    --dropout 0.3 --weight-decay 0.0 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 0.01 --lr-scheduler inverse_sqrt --warmup-updates 10 \
    --max-tokens 4096 --update-freq 16 \
    --max-update 1500 \
    --keep-last-epochs	10 \
    --save-dir trained_models

2024-07-01 16:21:01.103529: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 16:21:01.103586: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 16:21:01.105207: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
epoch 001:   0%|                                          | 0/3 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/n

In [26]:
ls trained_models

checkpoint492.pt  checkpoint495.pt  checkpoint498.pt  checkpoint501.pt
checkpoint493.pt  checkpoint496.pt  checkpoint499.pt  checkpoint_best.pt
checkpoint494.pt  checkpoint497.pt  checkpoint500.pt  checkpoint_last.pt


In [27]:
! python fairseq/fairseq_cli/interactive.py data-bin/trial \
  -s tu -t kn \
  --distributed-world-size 1 \
  --path trained_models/checkpoint_best.pt \
  --batch-size 64 --buffer-size 2500 --beam 10 \
  --skip-invalid-size-inputs-valid-test \
  --input valid.tu > valid.kn.translated

2024-07-01 16:37:31.917575: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 16:37:31.917629: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 16:37:31.919319: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [28]:
from sacrebleu import corpus_bleu

# Step 1: Read the translated output
translated_output_file = "valid.kn.translated" 
translated_output = []
with open(translated_output_file, "r") as f:
    for line in f:
        translated_output.append(line.strip())

# Step 2: Read the reference translations
reference_file = "valid.kn"  
reference_translations = []
with open(reference_file, "r") as f:
    for line in f:
        reference_translations.append(line.strip())

# Step 3: Tokenize the translated output and reference translations
tokenized_output = [tokenizer.tokenize(sentence).lower() for sentence in translated_output]
tokenized_references = [[tokenizer.tokenize(sentence).lower()] for sentence in reference_translations]

# Step 4: Compute the BLEU score
bleu_score = corpus_bleu(tokenized_output, tokenized_references)
print("BLEU score:", bleu_score.score)


BLEU score: 25.04359214460493
